In [4]:
import pandas as pd

In [5]:
data = pd.read_csv('edit_tabnak_news.csv')

In [5]:
data

,title,lead,category,id_news
0,حضور طالب لو در سپاهان صحت ندارد,برخي رسانه ها امروز و در چند روز گذشته خبر از ...,ورزشی,100000
1,نخستين پيروزي بسكتبال مهرام در آسیا,مهرامي ها در كوارترهاي اول و سوم را با نتايج (...,ورزشی,100001
2,تبريک حزب الله لبنان بابت قهرماني سپاهان,محمد رضا ساکت نيز در نامه هاي جداگانه اي به سي...,ورزشی,100003
3,علي دايي: مطمئن باشيد تيم فصل آينده با نظر من ...,خب شما برويد از همان خبرگزاري بپرسيد. در كنفرا...,ورزشی,100004
4,چکیده مهمترین حوادث و اتفاقات شبانه روز گذشته,مرد شرور با 7 نوچه اش تحت تعقيب پليس قرار گرفت...,اجتماعی,100006
...,...,...,...,...
7741,برنامه كامل رقابت ورزشكاران ايراني در سنگاپور,برنامه مسابقات ورزشكاران ايراني در اولين دوره ...,ورزشی,113945
7742,نتایج هفته دوم لیگ برتر فوتسال؛ اولین پیروزی پ...,هفته دوم لیگ برتر فوتسال با انجام 6 بازی به پا...,ورزشی,113947
7743,از تغییر خودسرانه دز دارو بپرهیزید,وی تصریح کرد: ما گاهی اوقات شاهد هستیم که افرا...,اجتماعی,113949
7744,داستان مترو تهران از خطبه آیت الله هاشمی تا نپ...,گرچه بارها دولت با فرافکنی مشکل مترو تهران را ...,اجتماعی,113950


In [6]:
import nltk
with open("stopwords.txt", encoding='utf-8') as stopwords_file:
    stopwords = stopwords_file.readlines()
stopwords = [stopword.replace('\n', '') for stopword in stopwords]

nltk.download('stopwords')
nltk_stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(nltk_stopwords)
len(stopwords)

In [1]:
# !pip install hazm

In [9]:
import pandas as pd
from hazm import word_tokenize, Stemmer, Lemmatizer
stemmer = Stemmer()
lem = Lemmatizer()
dataset = pd.DataFrame(columns=('title_lead', 'category'))
for index, row in data.iterrows():
    title_lead = row['title']+' '+row['lead']
    title_lead_tokenized = word_tokenize(title_lead)
    title_lead_filtered = [
        w for w in title_lead_tokenized if not w in stopwords]
    title_lead_filtered_stemmed = [
        stemmer.stem(w) for w in title_lead_filtered]
    title_lead_filtered_lemmatized = [
        lem.lemmatize(w).replace('#', ' ') for w in title_lead_filtered_stemmed]
    dataset.loc[index] = {
        "title_lead": ' '.join(title_lead_filtered_stemmed)+' '+' '.join(title_lead_filtered_lemmatized),
        'category': row['category']
    }

In [ ]:
# dataset.head()
dataset.to_csv('clean_tabnak_news.csv')

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [11]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
vectorizer.fit(dataset['title_lead'])
X = vectorizer.transform(dataset['title_lead'])

In [12]:
le = LabelEncoder()
y = le.fit_transform(dataset['category'])
len(y)

7746

In [13]:
import numpy as np
np.shape(X)

(7746, 214194)

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
from sklearn.naive_bayes import GaussianNB

In [19]:
print(X)

  (0, 190142)	0.18310789316509096
  (0, 190135)	0.14849827596313728
  (0, 186836)	0.19149092168266152
  (0, 186814)	0.1335311871747598
  (0, 148329)	0.19149092168266152
  (0, 148320)	0.19149092168266152
  (0, 148304)	0.26015465712451796
  (0, 136516)	0.14494525604660308
  (0, 136456)	0.06643651975084558
  (0, 135143)	0.16877701125858288
  (0, 135127)	0.0905385414552356
  (0, 124896)	0.3055824779726751
  (0, 124878)	0.2634644378559925
  (0, 122834)	0.19149092168266152
  (0, 122820)	0.13231465440827384
  (0, 113849)	0.19149092168266152
  (0, 113796)	0.19149092168266152
  (0, 113702)	0.20850442118889365
  (0, 96323)	0.16558993173150785
  (0, 96253)	0.09459192946569558
  (0, 90216)	0.04194880165782988
  (0, 86029)	0.15279123898633756
  (0, 85997)	0.11278782351127026
  (0, 81629)	0.09574546084133076
  (0, 81628)	0.08019699137050619
  :	:
  (7745, 119350)	0.10437247425054298
  (7745, 118095)	0.1658718320484276
  (7745, 118094)	0.1658718320484276
  (7745, 118093)	0.1658718320484276
  (7745, 1

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [17]:
X_train = pd.DataFrame(X_train.toarray())
X_test = pd.DataFrame(X_test.toarray())

In [18]:
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB()

In [20]:
from sklearn.metrics import classification_report, confusion_matrix

In [21]:
from sklearn.metrics import accuracy_score

In [ ]:
# y_pred_train = model.predict(X_train)
# y_pred_test = model.predict(X_test)

In [22]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.75      0.74       582
           1       0.54      0.17      0.26        83
           2       0.63      0.74      0.68       363
           3       0.56      0.27      0.37       184
           4       0.85      0.95      0.90       725

    accuracy                           0.75      1937
   macro avg       0.66      0.58      0.59      1937
weighted avg       0.74      0.75      0.73      1937



In [23]:
confusion_matrix(y_test, y_pred)

array([[436,   9,  69,  18,  50],
       [ 41,  14,  17,   1,  10],
       [ 47,   1, 269,  14,  32],
       [ 49,   1,  58,  50,  26],
       [ 17,   1,  15,   6, 686]])

##4:"ورزشی"

##3:"فرهنگی"

##2:"سیاسی"

##1:"اقتصادی"

##0:'اجتماعی'
